<a href="https://colab.research.google.com/github/cipriangerea/quant/blob/main/StrategyAllocationVis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import plotly.express as px
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
from io import StringIO
import requests
import json
import pdb



In [ ]:
# borrowed from https://github.com/howardr/polyphony/blob/main/src/composer.py

def fetch_backtest(id, start_date, end_date):
  if id.endswith('/details'):
    id = id.split('/')[-2]
  else :
    id = id.split('/')[-1]

  payload = {
    "capital": 100000,
    "apply_reg_fee": True,
    "apply_taf_fee": True,
    "backtest_version": "v2",
    "slippage_percent": 0.0005,
    "start_date": start_date,
    "end_date": end_date,
  }

  url = f"https://backtest-api.composer.trade/api/v2/public/symphonies/{id}/backtest"

  data = requests.post(url, json=payload)
  jsond = data.json()
  symphony_name = jsond['legend'][id]['name']

  holdings = jsond["last_market_days_holdings"]

  tickers = []
  for ticker in holdings:
    tickers.append(ticker)

  # Example format
  # {
  #   // key: ticker
  #   "SPY": {
  #     // key: days since linux epoch
  #     // value: percent allocation of ticker on date
  #     "19416": 0.123
  #   }
  # }
  allocations = jsond["tdvm_weights"]
  date_range = pd.date_range(start=start_date, end=end_date)
  df = pd.DataFrame(0.0, index=date_range, columns=tickers)

  for ticker in allocations:
    for date_int in allocations[ticker]:
      trading_date = convert_trading_date(date_int)
      percent = allocations[ticker][date_int]
      df.at[trading_date, ticker] = percent

  return df,symphony_name


def convert_trading_date(date_int):
  date_1 = datetime.strptime("01/01/1970", "%m/%d/%Y")
  dt = date_1 + timedelta(days=int(date_int))

  return dt


In [ ]:
url = input("Enter symphony url (leave blank for default):")
if not url:
    url =  'https://app.composer.trade/symphony/1so6GqqB7GgkUqfd2pFM/details'
df,symphony_name = fetch_backtest(url, '2000-01-01', '2024-05-09')


# Finding the first row with at least one non-zero value
first_valid_index = df[(abs(df) > 0.000001).any(axis=1)].first_valid_index()
print('Backtest for ', symphony_name, ' starting on ', first_valid_index)

# get rid of data prior to start of backtest and non-trading days
df = df.loc[(df != 0).any(axis=1)] * 100.0


In [ ]:
# # Upload files from your local system
# uploaded = files.upload()

# # Assuming a single file is uploaded, load it into a DataFrame
# filename = next(iter(uploaded))

In [ ]:
# #load into dataframe
# if 'Portfolio Visualizer' in filename:
#   # this is fron QuantMage
#   df = pd.read_csv(filename, parse_dates=['Start Date'])
#   rows = []

#   for _, row in df.iterrows():
#       date = row['Start Date']
#       assets = row['Assets'].split(', ')
#       weights = [float(weight.replace('%', '')) for weight in row['Weights'].split(', ')]
#       for asset, weight in zip(assets, weights):
#         rows.append({'Start Date': date, 'Asset': asset, 'Weight': weight})

#   # Convert the list to DataFrame
#   expanded_df = pd.DataFrame(rows)
#   expanded_df.rename(columns={'Start Date': 'Date'}, inplace=True)
#   df = expanded_df.pivot(index='Date', columns='Asset', values='Weight').fillna(0)

#   # whatever doesn't add to 100% goes into $USD
#   if 'BRK/B' in df.columns:
#     df.rename(columns={'BRK/B': 'BRK-B'}, inplace=True)

#   columns_to_process = [col for col in df.columns if col not in ['Date']]
#   df['$USD'] = 0.0
#   for _, row in df.iterrows():
#     sum = 0.0
#     for col in columns_to_process:
#       sum += row[col]
#     row['$USD'] = 100.0 - sum

# else:
#   df = pd.read_csv(filename, parse_dates=['Date'])
#   df.drop('Day Traded', axis=1, inplace=True)
#   df.set_index('Date', inplace=True)

#   if 'BRK/B' in df.columns:
#       df.rename(columns={'BRK/B': 'BRK-B'}, inplace=True)
#   # clean the data
#   # Exclude 'Date' and 'Day Traded' from the columns to be processed
#   columns_to_process = [col for col in df.columns if col not in ['Date', 'Day Traded']]

#   # Process the data: convert '-' to 0 and percentages to floats
#   for col in columns_to_process:
#       df[col] = df[col].replace('-', '0%').str.rstrip('%').astype('float')

#   if '$USD' in df.columns:
#         # If column exists, fill missing values with 0
#         df['$USD'].fillna(0, inplace=True)
#   else:
#         # If column does not exist, create it and fill with 0s
#         df['$USD'] = 0

columns_to_process = [col for col in df.columns if col not in ['Date', 'Day Traded']]
if 'BRK/B' in df.columns:
  df.rename(columns={'BRK/B': 'BRK-B'}, inplace=True)

if '$USD' in df.columns:
  # If column exists, fill missing values with 0
  df['$USD'].fillna(0.0, inplace=True)
else:
  # If column does not exist, create it and fill with 0s
  df['$USD'] = 0


In [ ]:
# let us use yfinance to compute profit stats
# Extract unique tickers

tickers = df.columns
unique_tickers = {ticker for ticker in tickers if ticker != '$USD'}  # Assume '$USD' is not a ticker

start_date = df.index.min()
end_date = df.index.max() + timedelta(days=1)  # Adding 3 days buffer

# Fetch historical prices
prices_tab = yf.download(list(unique_tickers), start=start_date, end=end_date)
prices = prices_tab['Adj Close'].copy()
volumes = prices_tab['Volume'].copy()
prices['$USD'] = 1.0
volumes['$USD'] = 0.0
prices=prices[tickers]
volumes=volumes[tickers]


In [ ]:
df.sort_index(inplace=True)
prices.sort_index(inplace=True)
volumes.sort_index(inplace=True)


# Initialize the portfolio value DataFrame
portfolio_values = pd.Series(index=df.index, dtype=float)
portfolio_values.iloc[0] = 100000  # Starting with $100,000

# Initialize DataFrame for tracking individual stock values
stock_gains_total_df = pd.DataFrame(index=df.index, columns=prices.columns, dtype=float)


# Iterate over each trading day
for i in range(1, len(df)):
    # Previous day's total portfolio value
    previous_total_value = portfolio_values.iloc[i-1]

    # Today's allocations and prices
    allocations_today = df.iloc[i, :]/100
    #pdb.set_trace()
    prices_today = prices.loc[df.index[i]]

    # Yesterday's allocations & prices
    allocations_yday = df.iloc[i-1, :]/100
    prices_yday = prices.loc[df.index[i-1]]

    # Calculate the money allocated to each ticker at end of yesterday
    money_allocated_yday = previous_total_value * allocations_yday

    # Calculate shares held for each ticker today (until right before close)
    shares_today = money_allocated_yday.divide(prices_yday, fill_value=0)

    # Estimate the total value at the end of today
    portfolio_values.iloc[i] = (shares_today * prices_today).sum()

    # an estimate of the gains, assuming that yesterday's holding
    # are held at today's closing prices
    gains_today = (prices_today - prices_yday) * shares_today
    stock_gains_total_df.iloc[i] = gains_today

# Display the portfolio value
print(portfolio_values.tail())

In [ ]:
# Plotting the portfolio value

# Assuming 'portfolio_values' DataFrame has a column named 'Value' and the index is 'Date'
fig = px.line(portfolio_values, x=portfolio_values.index, y=portfolio_values, title='Dynamic Portfolio Value Over Time')

# Update layout and axes titles
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
)

# Adding grid lines - Plotly has gridlines by default, but here's how to make sure:
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Add title and format axes
fig.update_layout(
    hovermode="x unified",
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    # yaxis=dict(fixedrange=True)   # Prevents zooming on the y-axis
    yaxis=dict(),   # Prevents zooming on the y-axis
    yaxis_type="log",
    dragmode='pan'
)

# Show plot
fig.show()

In [ ]:
# Plot an area chart for the processed columns
fig = px.area(df, y=columns_to_process,
                title='Portfolio Allocation Over Time',
                labels={'value': 'Allocation', 'variable': 'Asset'})

# Customize hover information
fig.update_traces(
    hoverinfo="x+y+name",
    hovertemplate="%{y}<extra>%{data.name}</extra>"
)

# Add title and format axes
fig.update_layout(
    hovermode="x unified",
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    #xaxis=dict(fixedrange=False),  # Allows x-axis to be zoomable
    yaxis=dict(fixedrange=True)   # Prevents zooming on the y-axis
)

fig.show()

In [ ]:
# Fetch historical prices for SPY
spy_prices = yf.download(['SPY'], start=start_date, end=end_date)
spy_prices['PctGain'] = spy_prices['Adj Close'].pct_change()

prices_pct = prices.pct_change()
prices_diff = prices_pct.sub(spy_prices['PctGain'], axis=0)

prices_diff_filtered = prices_diff[stock_gains_total_df!=0]


In [ ]:
filtered_df = stock_gains_total_df[stock_gains_total_df!=0]

cnt = filtered_df.count()
sum = filtered_df.sum()
cnt_pos = stock_gains_total_df[stock_gains_total_df>0].count()
win_spy = prices_diff_filtered[prices_diff_filtered>0]

avg_alloc = df[df!=0].mean()
daily_vol = (prices*volumes)

stats_df = pd.DataFrame({
    'DaysTradedPct' : cnt / stock_gains_total_df.count() * 100.0,
    'AvgAlloc' : avg_alloc,
    'PctWin' : cnt_pos / cnt * 100.0,
    'PctWinOverSpy' : win_spy.count() / cnt * 100.0,
    'WinVsSpy' : prices_diff_filtered.mean() * 100.0,
    'Profits' : sum,
    'DailyVolMean' : daily_vol.mean(),
    'DailyVolP10' : daily_vol.quantile(.1),
}).sort_values(by=['Profits'], ascending=False)

stats_df['Profits'] = stats_df['Profits'].map(lambda x: f"${x:,.2f}")
stats_df['DailyVolMean'] = stats_df['DailyVolMean'].map(lambda x: f"${x:,.2f}")
stats_df['DailyVolP10'] = stats_df['DailyVolP10'].map(lambda x: f"${x:,.2f}")
stats_df['PctWin'] = stats_df['PctWin'].map(lambda x: f"{x:,.2f}%")
stats_df['PctWinOverSpy'] = stats_df['PctWinOverSpy'].map(lambda x: f"{x:,.2f}%")
stats_df['WinVsSpy'] = stats_df['WinVsSpy'].map(lambda x: f"{x:,.2f}%")
stats_df['DaysTradedPct'] = stats_df['DaysTradedPct'].map(lambda x: f"{x:,.2f}%")
stats_df['AvgAlloc'] = stats_df['AvgAlloc'].map(lambda x: f"{x:,.2f}%")


# Set the maximum number of rows to display
pd.set_option('display.max_rows', 200)
# To display all columns
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 200)  # Adjust this as needed based on your screen size
pd.set_option('display.expand_frame_repr', False)  # Prevent the DataFrame from being split across the console


print(stats_df)

# The table above is to be interpreted as follows

1.   **DaysTradedPct** - how many days this ticker was traded
2.   **AvgAlloc** - what was the average allocation on days traded.
3.   **PctWin** - when allocated, what percentage of time did it produce a winning day
4.   **PctWinOverSpy** - when allocated, what percentage of time did it do better than SPY
5.   **WinVsSpy** - average win over SPY. Note this takes + and - days, so can be negative.
6.   **Profits** - how much money will it make in total.



In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set Plotly to render for Google Colab
pio.renderers.default = 'colab'


# Create a dropdown to select the ticker
dropdown = widgets.Dropdown(
    options=sorted(tickers),
    value='$USD',
    description='Ticker:'
)

# Create an output widget to display charts
output = widgets.Output()

# Function to update the chart
def update_chart(change):
    with output:
        clear_output(wait=True)  # Clear the old graph
        ticker = change['new']
        print('Detailed statistics for ', ticker)

        stock_gains_total_df_ticker = stock_gains_total_df[ticker]
        prices_diff_filtered_ticker = prices_diff_filtered[ticker]
        df_ticker = df[ticker]
        prices_ticker = prices[ticker]
        volumes_ticker = volumes[ticker]

        filtered_df = stock_gains_total_df_ticker[stock_gains_total_df_ticker!=0].groupby(pd.Grouper(freq='Y'))

        cnt = filtered_df.count()
        sum = filtered_df.sum()
        cnt_pos = stock_gains_total_df_ticker[stock_gains_total_df_ticker>0].groupby(pd.Grouper(freq='Y')).count()
        win_spy = prices_diff_filtered_ticker[prices_diff_filtered_ticker>0].groupby(pd.Grouper(freq='Y')).count()

        avg_alloc = df_ticker[df_ticker!=0].groupby(pd.Grouper(freq='Y')).mean()
        daily_vol = (prices_ticker*volumes_ticker).groupby(pd.Grouper(freq='Y'))
        stats_ticker = pd.DataFrame({
            'DaysTradedPct' : cnt / stock_gains_total_df_ticker.groupby(pd.Grouper(freq='Y')).count() * 100.0,
            'AvgAlloc' : avg_alloc,
            'PctWin' : cnt_pos / cnt * 100.0,
            'PctWinOverSpy' : win_spy / cnt * 100.0,
            'WinVsSpy' : prices_diff_filtered_ticker.groupby(pd.Grouper(freq='Y')).mean() * 100.0,
            'Profits' : sum,
            'DailyVolMean' : daily_vol.mean(),
            'DailyVolP10' : daily_vol.quantile(.1),
        })

        # stats_ticker['Date'] = stats_ticker.index
        # stats_ticker = stats_ticker.reset_index(drop=True)
        # stats_ticker = stats_ticker.sort_values(by=['Date'], ascending=False)

        stats_ticker['Profits'] = stats_ticker['Profits'].map(lambda x: f"${x:,.2f}")
        stats_ticker['DailyVolMean'] = stats_ticker['DailyVolMean'].map(lambda x: f"${x:,.2f}")
        stats_ticker['DailyVolP10'] = stats_ticker['DailyVolP10'].map(lambda x: f"${x:,.2f}")
        stats_ticker['PctWin'] = stats_ticker['PctWin'].map(lambda x: f"{x:,.2f}%")
        stats_ticker['PctWinOverSpy'] = stats_ticker['PctWinOverSpy'].map(lambda x: f"{x:,.2f}%")
        stats_ticker['WinVsSpy'] = stats_ticker['WinVsSpy'].map(lambda x: f"{x:,.2f}%")
        stats_ticker['DaysTradedPct'] = stats_ticker['DaysTradedPct'].map(lambda x: f"{x:,.2f}%")
        stats_ticker['AvgAlloc'] = stats_ticker['AvgAlloc'].map(lambda x: f"{x:,.2f}%")

        print(stats_ticker[::-1])

# Observe changes in the dropdown value
dropdown.observe(update_chart, names='value')

# Display the UI components
display(dropdown, output)

# Trigger the initial display
update_chart({'new': dropdown.value})  # Ensure to pass a dict with 'new' key

In [ ]:
%matplotlib inline
# conditional install quantstats
try:
  import quantstats as qs
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
    %pip install quantstats

  import quantstats as qs

import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Liberation Mono'

# extend pandas functionality with metrics, etc.
qs.extend_pandas()

output = widgets.Output()
with output:
  qs.reports.html(portfolio_values, "SPY", output='report.html')

import IPython
IPython.display.HTML(filename='report.html')